# Imports

In [88]:
import pandas as pd
import copy
import numpy as np
import pickle

# TODO

Regarder pourquoi on a pas l'index dans le df_mean_test.pkl

# Define strategies

In [157]:
def calculate_best_confidence_max_value(df,df_specific_id):
    '''
    Calculate the best confidence score based on the strategy max value. 
    @params df: data where 1 row is an image where tracking and classification where made 
    @paramns df_specific_id: data filtered on a specific track id
    Best test: track_id == 1
    '''
    max_confidence_index = df_specific_id['confidence'].idxmax()
    new_label = df.iloc[max_confidence_index,df.columns.get_loc("subclass")]
    return df_specific_id,max_confidence_index,new_label

def calculate_best_confidence_most_frequent(df,df_specific_id):
    '''
    Calculate the best confidence score based on the strategy most frequent
    @params df: data where 1 row is an image where tracking and classification where made 
    @paramns df_specific_id: data filtered on a specific track id
    Best test: track_id == 39
    '''
    most_frequent_species = df_specific_id.subclass.mode()
    if most_frequent_species.size != 1:
        pass # TO DO: if two most values -> get the one with best confidence
    else:
        new_label = most_frequent_species[0]
    most_frequent_specie = df_specific_id[df_specific_id['subclass'] == new_label]
    max_confidence_index = most_frequent_specie['confidence'].idxmax()
    return df_specific_id,max_confidence_index,new_label

def calculate_best_confidence_avg_max(df,df_specific_id):
    '''
    Calculate the mean for each unique value fo subclass
    Return the specie name and the index
    '''
    avgs = df_specific_id.reset_index().groupby(['subclass','index']).agg({'confidence': ['mean']}).reset_index().set_index('index')
    avgs.columns = ['subclass','confidence']
    avgs.index.name = None
    avgs.to_pickle("avgs_test.pkl")
    print('pickle 1 saved')

    #first get label from avgmax
    df_mean = avgs.groupby(['subclass']).mean()
    df_mean.to_pickle("df_mean_test.pkl")
    print('pickle 2 saved')
    new_label = df_mean.confidence.idxmax().values[0]

    #second get the avg of the max confident label
    new_confidence_score = df_mean.loc[new_label].values[0]
    
    #third get the max confidence index
    max_confidence_index = avgs['confidence'].idxmax()
    
    #replace the avg max for each iteration of label
    #df.loc['Hemaris gracilis',2082] = 100

    print(new_label, new_confidence_score)
    print(max_confidence_index)

    return df_mean,0,new_label

# Main

In [158]:
def modify_label_confidence(df,df_specific_id):
    '''
    Modify the label and confidence depending on the strategies. 
    From the dataframe specific to a trackID, we extract the best choice by the classification algorithm for a specific choice strategy 
    Then for each row of the specific dataframe we extract the index and make changes in the base dataframe.
     
    '''
    df_specific_id_modified,max_confidence_index,new_label = calculate_best_confidence_avg_max(df,df_specific_id)
    for index, row in df_specific_id_modified.iterrows():
        if (index != max_confidence_index) :
            df.iloc[index, df.columns.get_loc("subclass")] = new_label
            df.iloc[index, df.columns.get_loc("confidence")] = 'smoothed'
        
def smoothing_confidence_with_tracking(df_raw):
    '''
    Use the tracking insect results to smooth the classification.
    For each trackID, we created a sub-dataframe based on the specific trackID. 
    Then modify labels and confidence based on a specific strategy with "modify_label_confidence()"
    #TODO: replace empty by nan
    '''
    df = df_raw.copy()
    #max_id_tracked = int(df['track_id'].max())
    max_id_tracked = 39 #To test a specific trackID
    for track_id in range(39,max_id_tracked+1):
        df_ids_raw = df[df['track_id'] == track_id]
        if df_ids_raw.shape[0] > 1:
            df_specific_id = df_ids_raw.astype({'confidence': int}) #necessary, otherwise: confidence = object instead of int
            modify_label_confidence(df,df_specific_id)
    return df

def count_species(csv):
    '''
    Count unique individual per species
    '''
    df = pd.read_csv(csv)
    df_count = df.groupby(['subclass']).size().sort_values(ascending=False)
    df_count.to_csv('count.txt',header=False)
    return df_count

def main():
    df_raw = pd.read_csv("~/code/moth_project/0_database/tracking/track_localize_classify_annotation-2022_05_13.csv")
    df_modified = smoothing_confidence_with_tracking(df_raw)
    return df_modified


In [159]:
df = main()

pickle 1 saved
pickle 2 saved


AttributeError: 'str' object has no attribute 'values'

In [10]:
df

,image,track_id,bb_topleft_x,bb_topleft_y,bb_botright_x,bb_botright_y,bb_centre_x,bb_centre_y,class,subclass,confidence
0,20220513203542-00-01.jpg,NaN,3422,958,3550,1106,3486,1032,nonmoth,nonmoth,95
1,20220513203542-00-01.jpg,NaN,620,971,744,1110,682,1040,nonmoth,nonmoth,94
2,20220513203542-00-01.jpg,NaN,2977,805,3095,933,3036,869,nonmoth,nonmoth,94
3,20220513203542-00-01.jpg,NaN,2151,0,2334,100,2242,50,nonmoth,nonmoth,75
4,20220513203542-00-01.jpg,NaN,1073,807,1192,933,1132,870,nonmoth,nonmoth,89
...,...,...,...,...,...,...,...,...,...,...,...
15512,20220513235946-00-114.jpg,NaN,282,260,381,412,331,336,nonmoth,nonmoth,95
15513,20220513235946-00-114.jpg,NaN,522,762,654,910,588,836,nonmoth,nonmoth,95
15514,20220513235946-00-114.jpg,NaN,2975,805,3095,933,3035,869,nonmoth,nonmoth,91
15515,20220513235946-00-114.jpg,NaN,1073,808,1190,934,1131,871,nonmoth,nonmoth,85


In [11]:
df.to_csv('test_most_frequent.csv')

# Test on pickle

In [140]:
df_ids_raw = df[df['track_id'] == 39]
df_ids_raw[['']]

,image,track_id,bb_topleft_x,bb_topleft_y,bb_botright_x,bb_botright_y,bb_centre_x,bb_centre_y,class,subclass,confidence
1994,20220513213720-00-23.jpg,39.0,1650,800,1738,926,1694,863,moth,test_label,smoothed
2006,20220513213722-00-23.jpg,39.0,1649,800,1738,925,1693,862,moth,test_label,smoothed
2017,20220513213725-00-23.jpg,39.0,1648,803,1738,927,1693,865,moth,test_label,smoothed
2027,20220513213727-00-23.jpg,39.0,1649,806,1737,926,1693,866,moth,test_label,smoothed
2034,20220513213747-00-24.jpg,39.0,901,388,995,495,948,441,moth,test_label,smoothed
2046,20220513213749-00-24.jpg,39.0,917,333,1009,443,963,388,moth,test_label,smoothed
2059,20220513213751-00-24.jpg,39.0,917,331,1009,444,963,387,moth,test_label,smoothed
2072,20220513213753-00-24.jpg,39.0,918,329,1009,444,963,386,moth,test_label,smoothed
2082,20220513213756-00-24.jpg,39.0,1315,1010,1409,1117,1362,1063,moth,test_label,smoothed
2095,20220513213758-00-24.jpg,39.0,1761,687,1848,798,1804,742,moth,test_label,smoothed


In [115]:
_ = pd.read_pickle("avgs.pkl")

In [116]:
_

,,confidence
,,mean
subclass,index,
Acleris variana,2027,17.0
Alypia octomaculata,2103,15.0
Archips argyrospila,1994,16.0
Arctia parthenos,2072,64.0
Argyrotaenia velutinana,2116,62.0
Catocala ilia,2017,10.0
Euclea delphinii,2046,26.0
Eudryas unio,2127,25.0


In [118]:
_.loc['Hemaris gracilis',2082] = 100
print(_)

                              confidence
                                    mean
subclass                index           
Acleris variana         2027        17.0
Alypia octomaculata     2103        15.0
Archips argyrospila     1994        16.0
Arctia parthenos        2072        64.0
Argyrotaenia velutinana 2116        62.0
Catocala ilia           2017        10.0
Euclea delphinii        2046        26.0
Eudryas unio            2127        25.0
Haploa contigua         2095         8.0
Hemaris gracilis        2034        45.0
                        2082       100.0
Phobetron pithecium     2006        14.0
Thyris maculata         2059        40.0


In [121]:
df_mean = _.groupby(['subclass']).mean()
print(df_mean)
new_label = df_mean.confidence.idxmax().values[0]
print(new_label)

                        confidence
                              mean
subclass                          
Acleris variana               17.0
Alypia octomaculata           15.0
Archips argyrospila           16.0
Arctia parthenos              64.0
Argyrotaenia velutinana       62.0
Catocala ilia                 10.0
Euclea delphinii              26.0
Eudryas unio                  25.0
Haploa contigua                8.0
Hemaris gracilis              72.5
Phobetron pithecium           14.0
Thyris maculata               40.0
Hemaris gracilis


In [127]:
print(new_label)
print()

Hemaris gracilis


In [137]:
new_confidence_score = df_mean.loc[new_label]
print(new_confidence_score.values[0])

72.5


In [138]:
index_confidence_score = _.loc[new_label].index.values
index_confidence_score

array([2034, 2082])

In [130]:
len(index_confidence_score)

2

In [131]:
if len(index_confidence_score) > 1:
    index_confidence_score = index_confidence_score[0]

In [132]:
index_confidence_score

2034